# Python for data analytics – Project assessment

## Task 1 - Load your data

You should take your data from three files:
  * data/afl/stats.csv
  * data/afl/players.csv
  * data/afl/games.csv

And it should be loaded into a single dataframe by merging the three data sets.


In [ ]:
import pandas as pd

stats = pd.read_csv("data/afl/stats.csv")
players = pd.read_csv("data/afl/players.csv", index_col="playerId")
games = pd.read_csv("data/afl/games.csv", index_col="gameId")

games_stats = games.join(stats.set_index("gameId"), on='gameId', lsuffix='_games', rsuffix='_stats')
games_stats_players = games_stats.join(players, on='playerId', lsuffix='_stats', rsuffix='_players')

# loaded into a single dataframe by merging the three data sets
games_stats_players

## Task 2 – Plot one player
For a particular player - say "Franklin, Lance" - plot their accumlation of goals over time. The x-axis should be the number of games played and the y-axis should be the number of goals accumulated. Thus we expect to see a line that monotonically increases, but in what way will depend on the player's career.

In [ ]:
#todo

## Task 3 – Plot multiple players
In the one chart, plot the lines for the following players:
•	  "Franklin, Lance"
•	  "Papley, Tom"
•	  "Mumford, Shane"
•	  "Hooker, Cale".

Plot each in a different colour so they can be distinguished and add a legend.

In [ ]:
#todo

## Task 4 – Linear Regression
Create a second plot showing just "Franklin, Lance" and "Hooker, Cale" but include the linear regression line for each. In other words. as well as showing their actual cummulative goals over time, plot their predicted goals over time where the prediction is done via Linear Regression. Be sure to use different colours for each line and include a legend.

In [ ]:
#todo